## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 5
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_5


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 17.5 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Baltimore Ravens,5,2021,27.2,9.1,291.8,104.4,13.6,148.8,23.4,...,12.7,93.2,18.6,11.6,11.0,0.4,1.743119,0.096491,1.6,2.666667
1,Dallas Cowboys,5,2021,34.0,8.4,266.8,117.3,11.3,172.8,23.4,...,12.1,79.4,16.6,12.4,12.2,1.8,2.212766,0.086538,2.6,4.333333


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
7,Buffalo Bills,2021,week_5,0.291505,0.261195,0.567679,0.428656,0.387259
5,Arizona Cardinals,2021,week_5,0.302893,0.294429,0.368001,0.338484,0.325952
2,Tampa Bay Buccaneers,2021,week_5,0.261487,0.255086,0.370216,0.300546,0.296834


In [7]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [8]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
7,Buffalo Bills,2021,week_5,0.291505,0.261195,0.567679,0.428656,0.387259
5,Arizona Cardinals,2021,week_5,0.302893,0.294429,0.368001,0.338484,0.325952
2,Tampa Bay Buccaneers,2021,week_5,0.261487,0.255086,0.370216,0.300546,0.296834
8,Los Angeles Rams,2021,week_5,0.312609,0.261876,0.266821,0.297253,0.284640
4,Cleveland Browns,2021,week_5,0.238547,0.154832,0.217146,0.241193,0.212930
1,Dallas Cowboys,2021,week_5,0.238393,0.175104,0.151848,0.270071,0.208854
3,Kansas City Chiefs,2021,week_5,0.218260,0.140345,0.217690,0.251688,0.206996
0,Baltimore Ravens,2021,week_5,0.197304,0.106770,0.205863,0.240102,0.187510
24,Cincinnati Bengals,2021,week_5,0.161112,0.105147,0.195333,0.249434,0.177757
6,Los Angeles Chargers,2021,week_5,0.226072,0.132304,0.132458,0.216342,0.176794
